# Characterizing DNS Censorship in Venezuela with OONI

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections
import datetime as dt
import enum
import gzip
import itertools
import ipaddress
import json
import posixpath
from pprint import pprint
import shutil
import sys
import tempfile
from typing import Iterable, NamedTuple, Tuple, Union
from urllib.parse import urlparse

import pandas as pd
import ujson
import IPython.display as idisplay
import ipywidgets as widgets

try:
    import netanalysis
except ModuleNotFoundError:
    !{sys.executable} -m ensurepip
    !{sys.executable} -m pip install --upgrade git+https://github.com/Jigsaw-Code/net-analysis.git

import netanalysis.ooni.bucket as ob
import netanalysis.ooni.analysis.dns as od
from netanalysis.ooni.measurement import Measurement

## Fetch and save measurement data

In [3]:
import pathlib

COUNTRY = 'VE'
DATA_DIR = pathlib.Path('/kaggle/working')
if not DATA_DIR.is_dir():
    DATA_DIR = pathlib.Path.home()
DATA_DIR /= 'ooni_data'
local_measurements = ob.LocalMeasurements(DATA_DIR)

In [7]:
%%time
FIRST_DATE = dt.date.today() - dt.timedelta(days=7)
LAST_DATE = dt.date.today()
bytes, cost = ob.sync_measurements(local_measurements, ob.Bucket().list_files(FIRST_DATE, LAST_DATE, None, COUNTRY), cost_usd_limit=1.00)
print(f'Download size: {bytes/2**30:0.6f} GiB\nEstimated Cost: ${cost:02f}')

nnectivity.n0.0.jsonl.gz [3,498,143 bytes]
Download size: 0.552128 GiB
Estimated Cost: $0.049691
CPU times: user 43.4 s, sys: 3.82 s, total: 47.2 s
Wall time: 6min 27s


In [10]:
%%time
FIRST_DATE = dt.date.today() - dt.timedelta(days=7)
LAST_DATE = dt.date.today()
bytes, cost = ob.sync_measurements(local_measurements, ob.Bucket().list_files(FIRST_DATE, LAST_DATE, None, COUNTRY), cost_usd_limit=1.00)
print(f'Download size: {bytes/2**30:0.6f} GiB\nEstimated Cost: ${cost:02f}')

nnectivity.n0.0.jsonl.gz [3,498,143 bytes]
Download size: 0.552128 GiB
Estimated Cost: $0.049691
CPU times: user 54.7 s, sys: 2.58 s, total: 57.3 s
Wall time: 5min 52s


In [13]:
%%time
FIRST_DATE = dt.date.today() - dt.timedelta(days=7)
LAST_DATE = dt.date.today()
bytes, cost = ob.sync_measurements(local_measurements, ob.Bucket().list_files(FIRST_DATE, LAST_DATE, None, COUNTRY), cost_usd_limit=1.00)
print(f'Download size: {bytes/2**30:0.6f} GiB\nEstimated Cost: ${cost:02f}')

ectivity/2020111806_VE_webconnectivity.n0.1.jsonl.gz [3,477,655 bytes]
Download size: 0.552131 GiB
Estimated Cost: $0.049692
CPU times: user 53.6 s, sys: 2.34 s, total: 55.9 s
Wall time: 4min 50s


In [11]:
!rm -rf $DATA_DIR

In [8]:
!(cd $DATA_DIR; du -d 2 -h)

 68K	./VE/facebookmessenger
 72K	./VE/httpinvalidrequestline
 29M	./VE/webconnectivity
 84K	./VE/psiphon
332K	./VE/tor
112K	./VE/telegram
 64K	./VE/dash
664K	./VE/ndt
 72K	./VE/httpheaderfieldmanipulation
168K	./VE/whatsapp
 31M	./VE
 31M	.


# DNS Analysis

In [20]:
def is_ip(hostname):
    try:
        ipaddress.ip_address(domain)
        return True
    except ValueError:
        return False

evaluator: od.Evaluator = od.Evaluator()
rows = []
for measurement in local_measurements.get_measurements(COUNTRY, 'webconnectivity'):
    m = Measurement(measurement)
    domain = m.hostname
    if is_ip(domain):
        continue
    evaluator.add_control(m)
    rows.extend([o._asdict() for o in od.get_observations(m)])

for row in rows:
    evaluation = evaluator.evaluate(row['domain'], row['status'], row['answers'])
    row['eval'] = evaluation
    row['has_interference'] = 1 if evaluation.startswith('BAD') else 0

obs = pd.DataFrame.from_records(rows).sort_values(by='time', ascending=False)
del(rows)
obs

,time,client_country,client_asn,resolver_ip,resolver_asn,domain,query_type,failure,status,answers,explorer_url,eval,has_interference
40136,2020-11-20 06:59:55+00:00,VE,21826,173.194.97.13,15169,www.privaterra.org,AAAA,None,OK,[2800:3f0:4005:402::2013],https://explorer.ooni.org/measurement/20201120...,INCONCLUSIVE_CHECK_IPS,0
40135,2020-11-20 06:59:55+00:00,VE,21826,173.194.97.13,15169,www.privaterra.org,A,None,OK,[216.58.222.211],https://explorer.ooni.org/measurement/20201120...,INCONCLUSIVE_CHECK_IPS,0
33971,2020-11-20 06:59:47+00:00,VE,8048,201.249.172.71,8048,www.ectaco.com,A,None,OK,[158.69.42.114],https://explorer.ooni.org/measurement/20201120...,OK_MATCHES_CONTROL_IP,0
33847,2020-11-20 06:59:46+00:00,VE,21826,173.194.97.13,15169,www.islamicity.com,AAAA,None,OK,"[2606:4700:20::ac43:490f, 2606:4700:20::681a:d...",https://explorer.ooni.org/measurement/20201120...,INCONCLUSIVE_CHECK_IPS,0
33846,2020-11-20 06:59:46+00:00,VE,21826,173.194.97.13,15169,www.islamicity.com,A,None,OK,"[172.67.73.15, 104.26.12.124, 104.26.13.124]",https://explorer.ooni.org/measurement/20201120...,OK_MATCHES_CONTROL_IP,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48099,2020-11-13 03:00:48+00:00,VE,8048,201.249.172.71,8048,www.broadvoice.com,A,None,OK,"[141.193.213.21, 141.193.213.20]",https://explorer.ooni.org/measurement/20201113...,OK_MATCHES_CONTROL_IP,0
48134,2020-11-13 03:00:31+00:00,VE,8048,201.249.172.71,8048,imp.free.fr,A,None,OK,[212.27.48.1],https://explorer.ooni.org/measurement/20201113...,OK_MATCHES_CONTROL_IP,0
48139,2020-11-13 03:00:29+00:00,VE,21826,172.253.242.37,15169,extratorrent.cc,AAAA,None,OK,"[2606:4700:3031::681c:1232, 2606:4700:3036::68...",https://explorer.ooni.org/measurement/20201113...,INCONCLUSIVE_CHECK_IPS,0
48138,2020-11-13 03:00:29+00:00,VE,21826,172.253.242.37,15169,extratorrent.cc,A,None,OK,"[104.28.18.50, 172.67.131.33, 104.28.19.50]",https://explorer.ooni.org/measurement/20201113...,OK_MATCHES_CONTROL_IP,0


In [21]:
top_domains = obs.groupby(by='domain')['has_interference'].sum().sort_values(ascending=False)
top_ases = obs.value_counts(subset='client_asn')
top_statuses = obs.value_counts(subset='status')
top_evals = obs.value_counts(subset='eval')

display_columns = []
for d in [top_domains[:30], top_ases, top_statuses, top_evals]:
    w = widgets.Output(layout=widgets.Layout(margin='10px'))
    w.append_display_data(d)
    display_columns.append(w)
widgets.HBox(display_columns, layout=widgets.Layout(background='black'))

In [22]:
pivot = pd.pivot_table(obs, index=['domain'], columns=['client_asn', 'eval'], values=[], aggfunc=len, fill_value=0)
# pivot = pivot.reindex(top_sites.index).sort_index(axis='columns', level=1, ascending=False, key=lambda l: top_statuses[l])[top_ases.index]
pivot = pivot.reindex(top_domains.index).reindex(pd.MultiIndex.from_product([top_ases.index, top_evals.index]), axis='columns', fill_value=0)
with pd.option_context('display.max_rows', 150, 'display.max_columns', 24):
    display(pivot.iloc[:100, :24])

client_asn                                     21826                         \
eval                           OK_MATCHES_CONTROL_IP INCONCLUSIVE_CHECK_IPS   
domain                                                                        
www.webwombat.com.au                               3                      0   
psiphon.ca                                         5                      5   
www.coinbase.com                                   5                      5   
www.tunnelbear.com                                 5                      0   
occupystreams.org                                 10                      0   
www.teenhealthfx.com                              15                      0   
www.wikispaces.com                                 0                     10   
www.tripod.lycos.com                              15                      0   
archive.is                                         0                     12   
www.helvetas.ch                                   15                      0   
www.socom.mil                                     13                      0   
www.reddit.com                                    15                      0   
anonymouse.org                                    14                      0   
www.ifj.org                                       14                      0   
www.icftu.org                                     13                      0   
pastebin.com                                      15                     15   
www.weebly.com                                    15                      0   
6topoder.com                                       0                      0   
www.pridemedia.com                                15                      0   
sunoticiero.com                                    0                      0   
www.google.com                                     0                    178   
equal-eyes.org                                    14                      0   
wwf.panda.org                                     15                      0   
www.barnesandnoble.com                             0                     15   
vdebate.blogspot.com                               0                      0   
bnaibrith.org                                     15                      0   
www.gamingday.com                                 15                      0   
gawker.com                                         0                     15   
www.gatesfoundation.org                            0                     14   
freedns.afraid.org                                13                      0   
experience.arcgis.com                             15                      0   
www.gearthblog.com                                15                      0   
weblogs.us                                        15                      0   
www.washingtontimes.com                           15                      0   
witness.org                                       15                      0   
www.slotland.com                                   0                     15   
www.maduradas.com                                  0                      0   
www.sexualhealth.com                              14                      0   
wire.com                                          15                      0   
www.frontlinedefenders.org                        15                      0   
www.43things.com                                  15                      0   
www.elfarandi.com                                  0                      0   
www.anonymizer.com                                15                      0   
www.arabtimes.com                                 15                      0   
www.angelfire.lycos.com                           15                      0   
www.altpenis.com                                  14                      0   
www.euthanasia.cc                                 15                      0   
www.alqassam.ps                                   

In [49]:
with pd.option_context('display.max_colwidth', 300):
    display(obs[obs['domain'] == 'psiphon.ca'].drop(columns=['domain']).sort_values(by=['client_asn', 'resolver_asn', 'resolver_ip', 'time']))

,time,client_country,client_asn,resolver_ip,resolver_asn,query_type,failure,status,answers,explorer_url,eval,has_interference
35381,2020-11-20 06:03:25+00:00,VE,6306,186.166.131.54,6306,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201120T050056Z_webconnectivity_VE_6306_n1_SumX5KILeq1HhyUw?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_NXDOMAIN,1
35382,2020-11-20 06:03:25+00:00,VE,6306,186.166.131.54,6306,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201120T050056Z_webconnectivity_VE_6306_n1_SumX5KILeq1HhyUw?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_NXDOMAIN,1
874,2020-11-19 05:08:33+00:00,VE,6306,186.166.131.56,6306,A,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201119T050305Z_webconnectivity_VE_6306_n1_mPnQ2fz5N6JMQ6Mo?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_NXDOMAIN,1
875,2020-11-19 05:08:33+00:00,VE,6306,186.166.131.56,6306,AAAA,dns_nxdomain_error,NXDOMAIN,None,https://explorer.ooni.org/measurement/20201119T050305Z_webconnectivity_VE_6306_n1_mPnQ2fz5N6JMQ6Mo?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_NXDOMAIN,1
40746,2020-11-14 06:48:15+00:00,VE,8048,201.249.172.70,8048,A,unknown_failure: lookup psiphon.ca on [scrubbed]: server misbehaving,SERVFAIL,None,https://explorer.ooni.org/measurement/20201114T050317Z_webconnectivity_VE_8048_n1_1YC3ob63tMYlNnxo?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_SERVFAIL,1
40747,2020-11-14 06:48:15+00:00,VE,8048,201.249.172.70,8048,AAAA,unknown_failure: lookup psiphon.ca on [scrubbed]: server misbehaving,SERVFAIL,None,https://explorer.ooni.org/measurement/20201114T050317Z_webconnectivity_VE_8048_n1_1YC3ob63tMYlNnxo?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_SERVFAIL,1
22752,2020-11-16 06:42:19+00:00,VE,8048,201.249.172.71,8048,AAAA,unknown_failure: lookup psiphon.ca on [scrubbed]: server misbehaving,SERVFAIL,None,https://explorer.ooni.org/measurement/20201116T050338Z_webconnectivity_VE_8048_n1_9rJZI3Cdh9l0TIGT?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_SERVFAIL,1
22751,2020-11-16 06:42:19+00:00,VE,8048,201.249.172.71,8048,A,unknown_failure: lookup psiphon.ca on [scrubbed]: server misbehaving,SERVFAIL,None,https://explorer.ooni.org/measurement/20201116T050338Z_webconnectivity_VE_8048_n1_9rJZI3Cdh9l0TIGT?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_SERVFAIL,1
39107,2020-11-20 05:25:04+00:00,VE,8048,201.249.172.71,8048,AAAA,unknown_failure: lookup psiphon.ca on [scrubbed]: server misbehaving,SERVFAIL,None,https://explorer.ooni.org/measurement/20201120T050548Z_webconnectivity_VE_8048_n1_gcRDSQ6DIkl2SE0s?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_SERVFAIL,1
39106,2020-11-20 05:25:04+00:00,VE,8048,201.249.172.71,8048,A,unknown_failure: lookup psiphon.ca on [scrubbed]: server misbehaving,SERVFAIL,None,https://explorer.ooni.org/measurement/20201120T050548Z_webconnectivity_VE_8048_n1_gcRDSQ6DIkl2SE0s?input=https%3A%2F%2Fpsiphon.ca%2F,BAD_STATUS_SERVFAIL,1


In [59]:
!dig @200.35.192.12 torproject.org


; <<>> DiG 9.10.6 <<>> @200.35.192.12 torproject.org
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 50158
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 5, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;torproject.org.			IN	A

;; AUTHORITY SECTION:
torproject.org.		43963	IN	NS	nsp.dnsnode.net.
torproject.org.		43963	IN	NS	ns1.torproject.org.
torproject.org.		43963	IN	NS	ns3.torproject.org.
torproject.org.		43963	IN	NS	ns4.torproject.org.
torproject.org.		43963	IN	NS	ns5.torproject.org.

;; Query time: 63 msec
;; SERVER: 200.35.192.12#53(200.35.192.12)
;; WHEN: Fri Nov 20 12:34:30 EST 2020
;; MSG SIZE  rcvd: 144



In [24]:
obs[['client_asn', 'resolver_asn', 'resolver_ip']].value_counts()

client_asn  resolver_asn  resolver_ip    
22313       22313         200.35.192.12      3878
21826       15169         172.253.242.44     3827
8048        8048          201.249.172.71     3641
                          201.249.172.74     1991
                          201.249.172.72     1984
                          201.249.172.77     1962
                          201.249.172.70     1942
6306        6306          186.166.131.56     1942
8048        8048          201.249.215.7      1925
                          201.249.215.6      1922
21826       15169         172.253.242.41     1921
                          74.125.184.10      1919
                          172.253.7.65       1917
                          74.125.75.72       1915
                          74.125.77.75       1914
                          172.253.7.68       1913
                          172.253.7.13       1913
                          74.125.78.130      1913
                          172.253.242.105    1912
        

In [29]:
obs[['client_asn', 'resolver_asn', 'eval']].value_counts()

client_asn  resolver_asn  eval                            
21826       15169         OK_MATCHES_CONTROL_IP               16949
                          INCONCLUSIVE_CHECK_IPS              10915
8048        8048          OK_MATCHES_CONTROL_IP                9062
                          INCONCLUSIVE_CHECK_IPS               6219
22313       22313         OK_MATCHES_CONTROL_IP                2213
6306        6306          OK_MATCHES_CONTROL_IP                2112
22313       22313         INCONCLUSIVE_CHECK_IPS               1513
6306        6306          INCONCLUSIVE_CHECK_IPS               1441
21826       15169         OK_MATCHES_CONTROL_ERROR              430
                          INCONCLUSIVE_BAD_CONTROL              388
8048        8048          INCONCLUSIVE_BAD_CONTROL              234
                          OK_MATCHES_CONTROL_ERROR              234
                          BAD_STATUS_generic_timeout_error      140
                          BAD_STATUS_SERVFAIL            

In [352]:
!dig @201.249.172.77 eldolarparalelo.info


; <<>> DiG 9.10.6 <<>> @201.249.172.77 eldolarparalelo.info
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: SERVFAIL, id: 57965
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;eldolarparalelo.info.		IN	A

;; Query time: 277 msec
;; SERVER: 201.249.172.77#53(201.249.172.77)
;; WHEN: Sat Nov 14 02:29:30 EST 2020
;; MSG SIZE  rcvd: 49



In [353]:
!dig @201.249.172.77 example.com


; <<>> DiG 9.10.6 <<>> @201.249.172.77 example.com
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: REFUSED, id: 50572
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;example.com.			IN	A

;; Query time: 72 msec
;; SERVER: 201.249.172.77#53(201.249.172.77)
;; WHEN: Sat Nov 14 02:29:44 EST 2020
;; MSG SIZE  rcvd: 40



In [27]:
for asn, asn_df in obs.groupby(by='client_asn'):
    print(f'======= AS{asn} ======')
    for result, result_df in asn_df.groupby(by='eval'):
        print(f'----- {result} ------')
        with pd.option_context('display.min_rows', 50):
            display(result_df['domain'].value_counts())
        print()
    print()
    

======= AS6306 ======
----- BAD_STATUS_NXDOMAIN ------


www.weebly.com        4
www.coinbase.com      4
www.tunnelbear.com    4
www.wikispaces.com    4
www.reddit.com        4
psiphon.ca            4
Name: domain, dtype: int64


----- BAD_STATUS_SERVFAIL ------


www.webwombat.com.au    4
occupystreams.org       2
Name: domain, dtype: int64


----- BAD_STATUS_generic_timeout_error ------


...

en.m.wikipedia.org              2
bitcoin.sipa.be                 1
bitbucket.org                   1
ondalasuperestacion.com         1
www.queerty.com                 1
www.mailinator.com              1
www.ea.com                      1
www.humanrightsconnected.org    1
parley.la                       1
Name: domain, dtype: int64


----- OK_MATCHES_CONTROL_IP ------


www.abortionno.org              1
bcv.org.ve                      1
www.unwatch.org                 1
www.queerty.com                 1
www.wcicc.org                   1
www.analitica.com               1
bitcoin.sipa.be                 1
www.godalone.org                1
phrack.org                      1
www.mombian.com                 1
peacefire.org                   1
www.mailinator.com              1
craigslist.org                  1
www.aguaclara.org               1
parley.la                       1
www.broadvoice.com              1
feministing.com                 1
www.mail.com                    1
www.ccel.org                    1
alt.com                         1
krishna.com                     1
www.magicjack.com               1
www.auduboninternational.org    1
bitbucket.org                   1
www.humanrightsconnected.org    1
accioncampesina.com             1
mastodon.xyz                    1
rarbg.to                        1
ondalasuperestacion.com         1
stackexchange.



======= AS269750 ======
----- INCONCLUSIVE_CHECK_IPS ------


www.infobae.com    1
Name: domain, dtype: int64